### Import cw-simulate module and create new instance

In [ ]:
import { SimulateCosmWasmClient } from "@oraichain/cw-simulate";

senderAddress = "orai14vcw5qk0tdvknpa38wz46js5g7vrvut8lk0lk6";
client = new SimulateCosmWasmClient({
  chainId: "Oraichain",
  bech32Prefix: "orai",
  metering: true
});


### Setup account balance

In [ ]:
import { coins } from '@cosmjs/stargate';

client.app.bank.setBalance(senderAddress, coins('10000000', 'orai'));

console.log(await client.getBalance(senderAddress, 'orai'));

### Deploy contract on SimulateCosmwasmClient

In [ ]:
import { Cw20BaseClient } from "@oraichain/common-contracts-sdk";

const wasmCode = new Uint8Array(await fetch("/wasm/cw20-base.wasm").then(res=>res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");
const { contractAddress } = await client.instantiate(senderAddress, codeId, {
    decimals: 6,
    symbol: "CVT",
    name: "cosmoverse token",
    mint: { minter: senderAddress },
    initial_balances: [{ address: senderAddress, amount: "1000000000" }] 
}, "cw20 token", "auto");

cw20Contract = new Cw20BaseClient(client, senderAddress, contractAddress);

console.log(await cw20Contract.tokenInfo());

### Setup IBC channels

In [ ]:
import { CWSimulateApp } from "@oraichain/cw-simulate";

cosmosChain = new CWSimulateApp({
    chainId: "cosmoshub-4",
    bech32Prefix: "cosmos"
});

// relay message between Cosmos Hub and Oraichain
cosmosChain.ibc.relay("channel-0", "transfer", "channel-0", "transfer", client.app);

### Send Orai token from Oraichain to Cosmos Hub

In [ ]:
// mint ibc/orai on cosmos hub and burn orai on oraichain
await client.app.ibc.sendTransfer({
    channelId: "channel-0",
    receiver: "cosmos1ur2vsjrjarygawpdwtqteaazfchvw4fgf0kulf",
    token: { amount: "1000000", denom: "orai" },
    sender: senderAddress,
    timeout: {
        timestamp: ""
    }
});

console.log(cosmosChain.bank.getBalance("cosmos1ur2vsjrjarygawpdwtqteaazfchvw4fgf0kulf"));

### Send back IBC Orai from Cosmos Hub to Oraichain

In [ ]:
// mint orai on oraichain and burn ibc/orai on cosmos hub
await cosmosChain.ibc.sendTransfer({
    channelId: "channel-0",
    receiver: senderAddress,
    token: { amount: "1000000", denom: "ibc/56c280c7163a5e37ea987d3fa195ea719b7b7c7d319615afaa17cba5a347cec4" },
    sender: "cosmos1ur2vsjrjarygawpdwtqteaazfchvw4fgf0kulf",
    timeout: {
        timestamp: ""
    }
});

console.log(cosmosChain.bank.getBalance("cosmos1ur2vsjrjarygawpdwtqteaazfchvw4fgf0kulf"));

### Setup port module using smart contract

In [ ]:
import { CwIcs20LatestClient } from "@oraichain/common-contracts-sdk";

// deploy ics20 contract 
const wasmCode = new Uint8Array(await fetch("/wasm/cw-ics20-latest.wasm").then(res=>res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");
const { contractAddress } = await client.instantiate(senderAddress, codeId, {
    allowlist: [],
    default_timeout: 3600,
    gov_contract: senderAddress,
    swap_router_contract: "placeholder"
}, "ics20", "auto");


ics20Contract = new CwIcs20LatestClient(client, senderAddress, contractAddress);

console.log(await ics20Contract.config());

### Setup IBC channel between Cosmos Hub transfer port module and Oraichain CosmWasm port module

In [ ]:
oraiPort = "wasm." + ics20Contract.contractAddress;
channel = "channel-2"

// init ibc channel between two chains
client.app.ibc.relay(channel, oraiPort, channel, "transfer", cosmosChain);

### Send Open and Connect packets

In [ ]:
import { IbcOrder, GenericError } from "@oraichain/cw-simulate";

await cosmosChain.ibc.sendChannelOpen({
    open_init: {
        channel: {
            counterparty_endpoint: {
                port_id: oraiPort,
                channel_id: channel
            },
            endpoint: {
                port_id: "transfer",
                channel_id: channel
            },
            order: IbcOrder.Unordered,
            version: "ics20-1",
            connection_id: "connection-0"
        }
    }
});

await cosmosChain.ibc.sendChannelConnect({
    open_ack: {
        channel: {
            counterparty_endpoint: {
                port_id: oraiPort,
                channel_id: channel
            },
            endpoint: {
                port_id: "transfer",
                channel_id: channel
            },
            order: IbcOrder.Unordered,
            version: "ics20-1",
            connection_id: "connection-0"
        },
        counterparty_version: "ics20-1"
    }
});

// handle IBC message timeout
cosmosChain.ibc.addMiddleWare((msg, app) => {
    const data = msg.data.packet;
    if (Number(data.timeout.timestamp) < cosmosChain.time) {
        throw new GenericError("timeout at " + data.timeout.timestamp);
    }
});

### Prepare for interaction

In [ ]:
import { coins } from "@cosmjs/stargate";

airiIbcDenom = "tron-testnet0x7e2A35C746F2f7C240B664F1Da4DD100141AE71F";

await ics20Contract.updateMappingPair({
   localAssetInfo: {
     token: {
       contract_addr: cw20Contract.contractAddress
     }
   },
   localAssetInfoDecimals: 6,
   denom: airiIbcDenom,
   remoteDecimals: 6,
   localChannelId: channel
});

// topup
client.app.bank.setBalance(ics20Contract.contractAddress, coins("10000000000000", "orai"));

console.log(await client.getBalance(ics20Contract.contractAddress, 'orai'));

### Send IBC token between Cosmos Hub and Oraichain

In [ ]:
import bech32 from "bech32";
import { toBinary } from "@cosmjs/cosmwasm-stargate";

const cosmosSenderAddress = bech32.encode("cosmos", bech32.decode(senderAddress).words);

// now send ibc package
const icsPackage = {
    amount: "100000000",
    denom: airiIbcDenom,
    receiver: senderAddress,
    sender: cosmosSenderAddress,
    memo: ""
};

// transfer from cosmos to oraichain, should pass
await cosmosChain.ibc.sendPacketReceive({
    packet: {
        data: toBinary(icsPackage),      
        src: {
          port_id: "transfer",
          channel_id: channel
        },
        dest: {
          port_id: oraiPort,
          channel_id: channel
        },
        sequence: 27,
        timeout: {
          block: {
            revision: 1,
            height: 12345678
          }
        }
    },
    relayer: cosmosSenderAddress
});

const transferBackMsg = {
    local_channel_id: channel,
    remote_address: cosmosSenderAddress,
    remote_denom: airiIbcDenom
};

await cw20Contract.send({
    amount: "100000000",
    contract: ics20Contract.contractAddress,
    msg: toBinary(transferBackMsg)
});
const ibcBalance = await cw20Contract.balance({ address: ics20Contract.contractAddress });

// should return initialBalance: 100000000
console.log("ibc balance", ibcBalance.balance);